In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import loguniform, randint, uniform

In [ ]:
df = pd.read_csv('hand_landmarks_data.csv')

In [ ]:
df.head()

In [ ]:
df.isna().sum().sum()

## No nulls

In [ ]:
df['label'].value_counts()

## Seems balanced

In [ ]:
unique_labels = df['label'].unique()
unique_labels

In [ ]:
choice = np.random.choice(unique_labels, size = 4)

random_rows = []
for label in choice:
    row = df[df['label'] == label].sample(1)
    random_rows.append(row)
    
random_rows = pd.concat(random_rows)
random_rows

In [ ]:
x_axis = [random_rows[f'x{i}'].values[0] for i in range(1, 22)]
y_axis = [random_rows[f'y{i}'].values[0] for i in range(1, 22)]
x_axis

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12,10))
for i in range(4):
    row = i // 2
    col = i % 2
    ax[row, col].scatter([random_rows[f'x{j}'].values[i] for j in range(1, 22)], 
                         [random_rows[f'y{j}'].values[i] for j in range(1, 22)], color = 'red')
    ax[row, col].set_title(random_rows.iloc[i, -1])
    ax[row, col].invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
## Preprocessing Steps
## First, normalize each landmark by the wrist (1)
## Second, normalize each landmark by dividing it on the bigger finger which is middle finger (12, but we are 1 based so 13)

df_processed = df.copy()
wrist_x = df_processed['x1'].copy()
wrist_y = df_processed['y1'].copy()

for i in range(1, 22):
    df_processed[f'x{i}'] = df_processed[f'x{i}'] - wrist_x
    df_processed[f'y{i}'] = df_processed[f'y{i}'] - wrist_y


mid_finger_tip_position = np.sqrt(df_processed['x13']**2 + df_processed['y13']**2)
for i in range(1, 22):
    df_processed[f'x{i}'] = df_processed[f'x{i}'] / mid_finger_tip_position
    df_processed[f'y{i}'] = df_processed[f'y{i}'] / mid_finger_tip_position


df_processed.head()

In [ ]:
df_processed.describe()

In [ ]:
choice = np.random.choice(unique_labels, size = 4)

random_rows = []
for label in choice:
    row = df_processed[df_processed['label'] == label].sample(1)
    random_rows.append(row)
    
random_rows = pd.concat(random_rows)
random_rows

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12,10))
for i in range(4):
    row = i // 2
    col = i % 2
    ax[row, col].scatter([random_rows[f'x{j}'].values[i] for j in range(1, 22)], 
                         [random_rows[f'y{j}'].values[i] for j in range(1, 22)], color = 'red')
    ax[row, col].set_title(random_rows.iloc[i, -1])
    ax[row, col].invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
y = df_processed['label']
X = df_processed.drop(columns = ['label'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
LR = LogisticRegression()
LR.fit(X_train, y_train)
LR.score(X_test, y_test)

# Trying different models

In [ ]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM": SVC(),
    "KNN": KNeighborsClassifier(),
    "Random Forest": RandomForestClassifier(),
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    results[name] = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred, average='weighted'),
        "Recall": recall_score(y_test, y_pred, average='weighted'),
        "F1 Score": f1_score(y_test, y_pred, average='weighted')
    }

results

In [ ]:
svm = SVC(class_weight='balanced')

param_dist = {
    'kernel': ['rbf'],
    'C': loguniform(1e-3, 1e3),
    'gamma': loguniform(1e-4, 1e1)
}

svm_random = RandomizedSearchCV(
    svm,
    param_distributions=param_dist,
    n_iter=40,
    cv=5,
    random_state=42,
)

svm_random.fit(X_train, y_train)

print("Best params:", svm_random.best_params_)
print("Test accuracy:", svm_random.score(X_test, y_test))

In [ ]:
knn = KNeighborsClassifier()

knn_param_dist = {
    'n_neighbors': randint(1, 30),
    'weights': ['uniform', 'distance'],     
    'p': [1, 2]                            
}

knn_random = RandomizedSearchCV(
    knn,
    param_distributions=knn_param_dist,
    n_iter=30,
    cv=5,
    random_state=42,
)

knn_random.fit(X_train, y_train)

print("KNN Best params:", knn_random.best_params_)
print("KNN Test accuracy:", knn_random.score(X_test, y_test))

In [ ]:
rf = RandomForestClassifier(class_weight='balanced', random_state=42)

rf_param_dist = {
    'n_estimators': randint(50, 500),
    'max_depth': randint(2, 20),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'bootstrap': [True, False]
}

rf_random = RandomizedSearchCV(
    rf,
    param_distributions=rf_param_dist,
    n_iter=30,
    cv=5, 
    random_state=42,
)

rf_random.fit(X_train, y_train)

print("RF Best params:", rf_random.best_params_)
print("RF Test accuracy:", rf_random.score(X_test, y_test))

In [ ]:
y_pred = svm_random.predict(X_test)
print("SVM accuracy : ",svm_random.score(X_test, y_test))
print("SVM f1_score : ",f1_score(y_test,y_pred,average="weighted"))
print("SVM recall_score : ",recall_score(y_test,y_pred,average="weighted")) 
print("SVM precision_score :",precision_score(y_test,y_pred,average="weighted"))

In [ ]:
y_pred = knn_random.predict(X_test)
print("KNN accuracy : ",knn_random.score(X_test, y_test))
print("KNN f1_score : ",f1_score(y_test,y_pred,average="weighted"))
print("KNN recall_score : ",recall_score(y_test,y_pred,average="weighted")) 
print("KNN precision_score :",precision_score(y_test,y_pred,average="weighted"))

In [ ]:
y_pred = rf_random.predict(X_test)
print("Random Forest accuracy : ",rf_random.score(X_test, y_test))
print("Random Forest f1_score : ",f1_score(y_test,y_pred,average="weighted"))
print("Random Forest recall_score : ",recall_score(y_test,y_pred,average="weighted")) 
print("Random Forest precision_score :",precision_score(y_test,y_pred,average="weighted"))

In [ ]:
import joblib
joblib.dump(svm_random,"svm_winner.pkl")

---

### Performance Before GridSearch

| Model               | Accuracy | Precision | Recall | F1-Score |
| ------------------- | -------- | --------- | ------ | -------- |
| Logistic Regression | 0.7864   | 0.7859    | 0.7864 | 0.7835   |
| SVM                 | 0.9320   | 0.9383    | 0.9320 | 0.9326   |
| KNN                 | 0.9550   | 0.9553    | 0.9550 | 0.9551   |
| Random Forest       | 0.9562   | 0.9565    | 0.9562 | 0.9562   |

---

### Performance After GridSearch

| Model         | Accuracy | Precision | Recall | F1-Score |
| ------------- | -------- | --------- | ------ | -------- |
| SVM           | 0.9846   | 0.9847    | 0.9846 | 0.9846   |
| KNN           | 0.9577   | 0.9581    | 0.9577 | 0.9577   |
| Random Forest | 0.9449   | 0.9454    | 0.9449 | 0.9449   |

---

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import joblib

classifier = joblib.load("svm_winner.pkl")

hand_module = mp.solutions.hands
drawer = mp.solutions.drawing_utils
detector = hand_module.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

camera = cv2.VideoCapture(0)

w = int(camera.get(3))
h = int(camera.get(4))

codec = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter("output.mp4", codec, 30, (w, h))

while camera.isOpened():
    success, img = camera.read()
    if not success:
        break

    img = cv2.flip(img, 1)

    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    detection = detector.process(rgb_img)

    if detection.multi_hand_landmarks:
        for hand_data in detection.multi_hand_landmarks:
            coords = np.array([(lm.x, lm.y, lm.z) for lm in hand_data.landmark])

            wx, wy, wz = coords[0]
            coords[:, 0] -= wx
            coords[:, 1] -= wy

            mx, my, _ = coords[12]
            sf = np.sqrt(mx**2 + my**2)
            coords[:, 0] /= sf
            coords[:, 1] /= sf

            feat = coords.flatten().reshape(1, -1)

            pred = classifier.predict(feat)[0]

            drawer.draw_landmarks(img, hand_data, hand_module.HAND_CONNECTIONS)

            cv2.putText(img, f'Prediction: {pred}', (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    writer.write(img)

    cv2.imshow("Hand Gesture Recognition", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
writer.release()
cv2.destroyAllWindows()